# 🎴 Card Recognition V4 - Curriculum Learning

**Progressive Training with Live Accuracy Monitoring**

- Phase 1: Clean cards, light augmentation
- Phase 2: Medium augmentation
- Phase 3: Full augmentation + backgrounds
- Live accuracy monitoring every 5 epochs

---

## 1️⃣ Setup

In [ ]:
!nvidia-smi
import torch
print(f"PyTorch: {torch.__version__}, CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
!pip install -q timm albumentations opencv-python-headless tqdm imagehash

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 2️⃣ Extract Data

In [ ]:
import os, zipfile, json, random
from pathlib import Path
from PIL import Image, ImageOps
from tqdm.notebook import tqdm
import numpy as np
import cv2

ZIP_PATH = "/content/drive/MyDrive/CardData/card_images.zip"
IMAGE_DIR = "/content/card_images"
CHECKPOINT_DIR = '/content/checkpoints'
DRIVE_OUTPUT = '/content/drive/MyDrive/CardRecognition_Models'
CARD_JSON = '/content/drive/MyDrive/CardData/card-flattened-with-phash.json'

for d in [CHECKPOINT_DIR, DRIVE_OUTPUT]:
    os.makedirs(d, exist_ok=True)

if os.path.exists(f"{IMAGE_DIR}/.extracted"):
    print(f"✓ Already extracted")
elif os.path.exists(ZIP_PATH):
    print("Extracting...")
    !rm -rf {IMAGE_DIR}
    os.makedirs(IMAGE_DIR, exist_ok=True)
    with zipfile.ZipFile(ZIP_PATH, 'r') as z:
        z.extractall(IMAGE_DIR)
    Path(f"{IMAGE_DIR}/.extracted").touch()
    print(f"✓ Done")

In [ ]:
if os.path.exists(f"{IMAGE_DIR}/.validated"):
    print("✓ Already validated")
else:
    print("Validating...")
    corrupted = []
    for p in tqdm(list(Path(IMAGE_DIR).glob('*'))):
        if p.suffix.lower() in ['.jpg','.jpeg','.png','.webp']:
            try:
                with Image.open(p) as img: img.verify()
                with Image.open(p) as img: img.load()
            except:
                corrupted.append(p.name)
                p.unlink()
    Path(f"{IMAGE_DIR}/.validated").touch()
    print(f"✓ Removed {len(corrupted)} corrupted")

## 3️⃣ Curriculum: Light → Medium → Full Augmentation

In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

def get_light_augmentations(size=224):
    """Phase 1: Minimal augmentation - learn basic card features."""
    return A.Compose([
        A.Resize(size, size),
        A.Rotate(limit=3, p=0.3),
        A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1, p=0.3),
        A.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
        ToTensorV2()
    ])

def get_medium_augmentations(size=224):
    """Phase 2: Moderate augmentation - add camera-like effects."""
    return A.Compose([
        A.Resize(size, size),
        A.Perspective(scale=(0.02, 0.05), p=0.3),
        A.Rotate(limit=8, p=0.4),
        A.OneOf([A.GaussianBlur(blur_limit=3), A.MotionBlur(blur_limit=3)], p=0.2),
        A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.4),
        A.GaussNoise(var_limit=(5, 20), p=0.2),
        A.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
        ToTensorV2()
    ])

def get_heavy_augmentations(size=224):
    """Phase 3: Heavy augmentation with backgrounds."""
    return A.Compose([
        A.Resize(size, size),
        A.Perspective(scale=(0.02, 0.08), p=0.5),
        A.Rotate(limit=15, border_mode=cv2.BORDER_CONSTANT, p=0.5),
        A.OneOf([A.MotionBlur(blur_limit=5), A.GaussianBlur(blur_limit=5)], p=0.3),
        A.RandomBrightnessContrast(brightness_limit=0.25, contrast_limit=0.25, p=0.5),
        A.RandomShadow(shadow_roi=(0, 0.3, 1, 1), p=0.2),
        A.GaussNoise(var_limit=(10, 40), p=0.3),
        A.ImageCompression(quality_lower=70, p=0.2),
        A.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
        ToTensorV2()
    ])

def get_val_transforms(size=224):
    return A.Compose([
        A.Resize(size, size),
        A.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
        ToTensorV2()
    ])

print("✓ Curriculum augmentations ready")

In [ ]:
class SyntheticBackground:
    def __init__(self, output_size=(480, 640)):
        self.output_size = output_size
    
    def solid_color(self):
        color = tuple(random.randint(0, 255) for _ in range(3))
        return np.full((*self.output_size, 3), color, dtype=np.uint8)
    
    def gradient(self):
        c1 = np.array([random.randint(0, 255) for _ in range(3)])
        c2 = np.array([random.randint(0, 255) for _ in range(3)])
        arr = np.zeros((*self.output_size, 3), dtype=np.uint8)
        for i in range(self.output_size[0]):
            t = i / self.output_size[0]
            arr[i] = (c1 * (1 - t) + c2 * t).astype(np.uint8)
        return arr
    
    def get_random_bg(self):
        return self.solid_color() if random.random() > 0.3 else self.gradient()
    
    def composite(self, card_img):
        bg = self.get_random_bg()
        h, w = card_img.shape[:2]
        scale = random.uniform(0.5, 0.8)
        new_h = int(self.output_size[0] * scale)
        new_w = int(new_h * w / h)
        card_resized = cv2.resize(card_img, (new_w, new_h))
        
        max_y = max(0, self.output_size[0] - new_h)
        max_x = max(0, self.output_size[1] - new_w)
        y = random.randint(0, max_y) if max_y > 0 else 0
        x = random.randint(0, max_x) if max_x > 0 else 0
        
        y_end = min(y + new_h, self.output_size[0])
        x_end = min(x + new_w, self.output_size[1])
        bg[y:y_end, x:x_end] = card_resized[:y_end-y, :x_end-x]
        return bg

synth_bg = SyntheticBackground()
print("✓ SyntheticBackground ready")

## 4️⃣ Model Architecture

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import timm

class GeM(nn.Module):
    def __init__(self, p=3.0, eps=1e-6):
        super().__init__()
        self.p = nn.Parameter(torch.ones(1) * p)
        self.eps = eps
    
    def forward(self, x):
        return F.adaptive_avg_pool2d(x.clamp(min=self.eps).pow(self.p), 1).pow(1./self.p).view(x.size(0), -1)

class ColorBranch(nn.Module):
    def __init__(self, bins=32, output_dim=64):
        super().__init__()
        self.bins = bins
        self.fc = nn.Sequential(nn.Linear(bins*3, 128), nn.ReLU(), nn.Dropout(0.3), nn.Linear(128, output_dim))
        self.register_buffer('mean', torch.tensor([0.485,0.456,0.406]).view(1,3,1,1))
        self.register_buffer('std', torch.tensor([0.229,0.224,0.225]).view(1,3,1,1))
    
    def forward(self, x):
        x_denorm = ((x * self.std + self.mean) * 255).clamp(0, 255)
        hists = []
        for i in range(x.shape[0]):
            img = x_denorm[i].permute(1,2,0).cpu().numpy().astype(np.uint8)
            hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
            h = np.histogram(hsv[:,:,0], bins=self.bins, range=(0,180))[0]
            s = np.histogram(hsv[:,:,1], bins=self.bins, range=(0,256))[0]
            v = np.histogram(hsv[:,:,2], bins=self.bins, range=(0,256))[0]
            hist = np.concatenate([h,s,v]).astype(np.float32)
            hists.append(hist / (hist.sum() + 1e-8))
        return self.fc(torch.tensor(np.stack(hists), device=x.device, dtype=torch.float32))

class CardEmbeddingNet(nn.Module):
    def __init__(self, embedding_dim=512, color_dim=64, pretrained=True):
        super().__init__()
        self.backbone = timm.create_model('mobilenetv3_small_100', pretrained=pretrained, num_classes=0, global_pool='')
        with torch.no_grad():
            self.num_features = self.backbone(torch.randn(1,3,224,224)).shape[1]
        self.gem = GeM()
        self.color_branch = ColorBranch(bins=32, output_dim=color_dim)
        self.fc = nn.Linear(self.num_features + color_dim, embedding_dim)
        self.bn = nn.BatchNorm1d(embedding_dim)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        visual = self.gem(self.backbone(x))
        color = self.color_branch(x)
        return F.normalize(self.dropout(self.bn(self.fc(torch.cat([visual, color], dim=1)))), p=2, dim=1)

print("✓ Model ready")

## 5️⃣ Dataset with Curriculum

In [ ]:
from torch.utils.data import Dataset, DataLoader

class CurriculumDataset(Dataset):
    def __init__(self, image_dir, rotations=[0, 90, 180, 270]):
        self.image_dir = Path(image_dir)
        self.rotations = rotations
        self.images = sorted([f for f in self.image_dir.iterdir() 
                              if f.suffix.lower() in ['.jpg','.jpeg','.png','.webp']])
        self.num_cards = len(self.images)
        self.samples = [(i, r) for i in range(len(self.images)) for r in rotations]
        
        # Curriculum settings (updated by trainer)
        self.transform = get_light_augmentations()
        self.use_backgrounds = False
        self.synth_bg = SyntheticBackground()
        
        print(f"Dataset: {self.num_cards} cards × {len(rotations)} rot = {len(self.samples)} samples")
    
    def set_phase(self, phase):
        """Update augmentation based on training phase."""
        if phase == 1:
            self.transform = get_light_augmentations()
            self.use_backgrounds = False
            print("📚 Phase 1: Light augmentation, no backgrounds")
        elif phase == 2:
            self.transform = get_medium_augmentations()
            self.use_backgrounds = False
            print("📚 Phase 2: Medium augmentation, no backgrounds")
        else:
            self.transform = get_heavy_augmentations()
            self.use_backgrounds = True
            print("📚 Phase 3: Heavy augmentation WITH backgrounds")
    
    def __len__(self): return len(self.samples)
    
    def __getitem__(self, idx):
        img_idx, rotation = self.samples[idx]
        try:
            img = np.array(Image.open(self.images[img_idx]).convert('RGB'))
            
            if rotation == 90: img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
            elif rotation == 180: img = cv2.rotate(img, cv2.ROTATE_180)
            elif rotation == 270: img = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)
            
            if self.use_backgrounds and random.random() > 0.5:
                img = self.synth_bg.composite(img)
            
            img = self.transform(image=img)['image']
            return img, img_idx
        except:
            return self.__getitem__(random.randint(0, len(self.samples)-1))
    
    def get_num_classes(self): return self.num_cards

print("✓ CurriculumDataset ready")

## 6️⃣ Live Accuracy Monitor

In [ ]:
class AccuracyMonitor:
    """Monitor actual identification accuracy during training."""
    
    def __init__(self, model, images, device, n_test=50):
        self.model = model
        self.images = images
        self.device = device
        self.n_test = n_test
        self.transform = get_val_transforms()
        self.reference_embeddings = None
        self.reference_names = None
    
    def build_references(self):
        """Build reference embeddings from all cards."""
        self.model.eval()
        embeddings = []
        names = []
        
        with torch.no_grad():
            for img_path in self.images:
                try:
                    img = np.array(Image.open(img_path).convert('RGB'))
                    tensor = self.transform(image=img)['image'].unsqueeze(0).to(self.device)
                    emb = self.model(tensor)
                    embeddings.append(emb.cpu())
                    names.append(img_path.stem)
                except:
                    pass
        
        self.reference_embeddings = torch.cat(embeddings, dim=0)
        self.reference_names = names
    
    def compute_accuracy(self):
        """Test identification accuracy on random subset."""
        if self.reference_embeddings is None:
            self.build_references()
        
        self.model.eval()
        test_indices = random.sample(range(len(self.images)), min(self.n_test, len(self.images)))
        
        top1, top5, top10 = 0, 0, 0
        
        with torch.no_grad():
            for idx in test_indices:
                img_path = self.images[idx]
                actual_name = img_path.stem
                
                try:
                    img = np.array(Image.open(img_path).convert('RGB'))
                    tensor = self.transform(image=img)['image'].unsqueeze(0).to(self.device)
                    query_emb = self.model(tensor).cpu()
                    
                    sims = F.cosine_similarity(query_emb, self.reference_embeddings)
                    top_indices = sims.argsort(descending=True)[:10]
                    top_names = [self.reference_names[i] for i in top_indices]
                    
                    if actual_name == top_names[0]: top1 += 1
                    if actual_name in top_names[:5]: top5 += 1
                    if actual_name in top_names[:10]: top10 += 1
                except:
                    pass
        
        n = len(test_indices)
        return {
            'top1': 100 * top1 / n,
            'top5': 100 * top5 / n,
            'top10': 100 * top10 / n
        }

print("✓ AccuracyMonitor ready")

## 7️⃣ Training Setup

In [ ]:
class CosFaceLoss(nn.Module):
    def __init__(self, num_classes, embedding_dim, scale=64.0, margin=0.4):
        super().__init__()
        self.scale, self.margin = scale, margin
        self.weight = nn.Parameter(torch.FloatTensor(num_classes, embedding_dim))
        nn.init.xavier_uniform_(self.weight)
    
    def forward(self, embeddings, labels):
        W = F.normalize(self.weight, p=2, dim=1)
        cosine = F.linear(embeddings, W)
        one_hot = torch.zeros_like(cosine).scatter_(1, labels.view(-1,1), 1.0)
        return F.cross_entropy((cosine - one_hot * self.margin) * self.scale, labels)

# Curriculum phases
PHASE_EPOCHS = {
    1: (1, 10),    # Epochs 1-10: Light
    2: (11, 20),   # Epochs 11-20: Medium  
    3: (21, 100),  # Epochs 21+: Heavy + backgrounds
}

CONFIG = {
    'epochs': 60,
    'batch_size': 64,
    'learning_rate': 1e-3,  # Higher for phase 1
    'weight_decay': 1e-4,
    'embedding_dim': 512,
    'patience': 15,
    'unfreeze_epoch': 8,
    'accuracy_check_interval': 5,  # Check accuracy every N epochs
}
print("✓ Config:", CONFIG)

In [ ]:
# Create dataset and loaders
train_ds = CurriculumDataset(IMAGE_DIR)
val_ds = CurriculumDataset(IMAGE_DIR, rotations=[0])
val_ds.transform = get_val_transforms()
val_ds.use_backgrounds = False

indices = np.random.permutation(train_ds.num_cards)
split = int(0.85 * train_ds.num_cards)
train_idx, val_idx = set(indices[:split]), set(indices[split:])

train_samples = [i for i, (c, _) in enumerate(train_ds.samples) if c in train_idx]
val_samples = [i for i, (c, _) in enumerate(val_ds.samples) if c in val_idx]

train_loader = DataLoader(torch.utils.data.Subset(train_ds, train_samples),
                          batch_size=CONFIG['batch_size'], shuffle=True, num_workers=2, 
                          pin_memory=True, drop_last=True)
val_loader = DataLoader(torch.utils.data.Subset(val_ds, val_samples),
                        batch_size=CONFIG['batch_size'], shuffle=False, num_workers=2, pin_memory=True)

num_classes = train_ds.get_num_classes()
print(f"✓ Train: {len(train_samples):,} | Val: {len(val_samples):,} | Classes: {num_classes:,}")

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CardEmbeddingNet(embedding_dim=CONFIG['embedding_dim']).to(device)

for p in model.backbone.parameters(): p.requires_grad = False
print("✓ Backbone frozen")

criterion = CosFaceLoss(num_classes, CONFIG['embedding_dim']).to(device)
optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()),
                              lr=CONFIG['learning_rate'], weight_decay=CONFIG['weight_decay'])
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=CONFIG['epochs'])
scaler = torch.amp.GradScaler('cuda')

# Accuracy monitor
accuracy_monitor = AccuracyMonitor(model, train_ds.images, device, n_test=100)

print("✓ Ready")

## 8️⃣ Training Loop with Live Monitoring

In [ ]:
best_loss = float('inf')
patience_counter = 0
history = {'train': [], 'val': [], 'top1': [], 'top5': []}
current_phase = 0
RESUME_PATH = f"{CHECKPOINT_DIR}/best_model.pth"

print("="*70)
print("TRAINING WITH CURRICULUM LEARNING")
print("="*70)

for epoch in range(1, CONFIG['epochs'] + 1):
    # Update curriculum phase
    for phase, (start, end) in PHASE_EPOCHS.items():
        if start <= epoch <= end and phase != current_phase:
            current_phase = phase
            train_ds.set_phase(phase)
            # Rebuild references when phase changes
            accuracy_monitor.reference_embeddings = None
            break
    
    # Unfreeze backbone
    if epoch == CONFIG['unfreeze_epoch']:
        print("\n🔓 Unfreezing backbone...")
        for p in model.backbone.parameters(): p.requires_grad = True
        optimizer = torch.optim.AdamW([
            {'params': model.backbone.parameters(), 'lr': CONFIG['learning_rate']/10},
            {'params': model.gem.parameters()},
            {'params': model.color_branch.parameters()},
            {'params': model.fc.parameters()},
            {'params': model.bn.parameters()},
        ], lr=CONFIG['learning_rate'], weight_decay=CONFIG['weight_decay'])
    
    # Training
    model.train()
    train_loss = 0
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch} [P{current_phase}]", leave=False):
        images, labels = images.to(device), labels.to(device)
        with torch.amp.autocast('cuda'):
            loss = criterion(model(images), labels)
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 5.0)
        scaler.step(optimizer)
        scaler.update()
        train_loss += loss.item()
    train_loss /= len(train_loader)
    
    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            val_loss += criterion(model(images), labels).item()
    val_loss /= len(val_loader)
    
    scheduler.step()
    history['train'].append(train_loss)
    history['val'].append(val_loss)
    
    # Live accuracy check
    if epoch % CONFIG['accuracy_check_interval'] == 0:
        acc = accuracy_monitor.compute_accuracy()
        history['top1'].append(acc['top1'])
        history['top5'].append(acc['top5'])
        print(f"Epoch {epoch}: Loss={train_loss:.2f}/{val_loss:.2f} | "
              f"Top-1: {acc['top1']:.1f}% | Top-5: {acc['top5']:.1f}% | Top-10: {acc['top10']:.1f}%")
    else:
        print(f"Epoch {epoch}: Train={train_loss:.4f}, Val={val_loss:.4f}")
    
    # Save best
    if val_loss < best_loss:
        best_loss = val_loss
        patience_counter = 0
        torch.save({'epoch': epoch, 'model_state_dict': model.state_dict(),
                    'val_loss': val_loss, 'num_classes': num_classes}, RESUME_PATH)
        print("  💾 Saved")
    else:
        patience_counter += 1
        if patience_counter >= CONFIG['patience']:
            print("\n⚠️ Early stopping!")
            break

print(f"\n✓ Done! Best loss: {best_loss:.4f}")

## 9️⃣ Results & Save

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(14, 4))

# Loss plot
axes[0].plot(history['train'], label='Train')
axes[0].plot(history['val'], label='Val')
axes[0].set_xlabel('Epoch'); axes[0].set_ylabel('Loss'); axes[0].legend()
axes[0].set_title('Training Loss')

# Accuracy plot
if history['top1']:
    x = list(range(CONFIG['accuracy_check_interval'], len(history['top1'])*CONFIG['accuracy_check_interval']+1, CONFIG['accuracy_check_interval']))
    axes[1].plot(x, history['top1'], 'o-', label='Top-1')
    axes[1].plot(x, history['top5'], 's-', label='Top-5')
    axes[1].set_xlabel('Epoch'); axes[1].set_ylabel('Accuracy %'); axes[1].legend()
    axes[1].set_title('Identification Accuracy')

plt.tight_layout()
plt.savefig(f"{CHECKPOINT_DIR}/training.png")
plt.show()

In [ ]:
import shutil
for f in ['best_model.pth', 'training.png']:
    src = f"{CHECKPOINT_DIR}/{f}"
    if os.path.exists(src):
        shutil.copy(src, DRIVE_OUTPUT)
        print(f"✓ Saved {f}")

## 🔟 Build References & Test

In [ ]:
print("Building final reference embeddings...")
ckpt = torch.load(RESUME_PATH)
model.load_state_dict(ckpt['model_state_dict'])
model.eval()

test_transform = get_val_transforms()
reference_embeddings = []
reference_names = []

with torch.no_grad():
    for img_path in tqdm(train_ds.images, desc="Building refs"):
        try:
            img = np.array(Image.open(img_path).convert('RGB'))
            emb = model(test_transform(image=img)['image'].unsqueeze(0).to(device))
            reference_embeddings.append(emb.cpu())
            reference_names.append(img_path.stem)
        except: pass

reference_embeddings = torch.cat(reference_embeddings, dim=0)
print(f"✓ {len(reference_embeddings):,} embeddings")

In [ ]:
# Final accuracy test
final_acc = accuracy_monitor.compute_accuracy()
print(f"\n{'='*50}")
print(f"FINAL ACCURACY")
print(f"{'='*50}")
print(f"Top-1:  {final_acc['top1']:.1f}%")
print(f"Top-5:  {final_acc['top5']:.1f}%")
print(f"Top-10: {final_acc['top10']:.1f}%")

In [ ]:
# Load card metadata for identifier
import imagehash

with open(CARD_JSON, 'r') as f:
    all_cards = json.load(f)

card_lookup = {c['printing_unique_id']: c for c in all_cards}
name_to_printings = {}
for card in all_cards:
    name = card.get('name', '')
    if name not in name_to_printings:
        name_to_printings[name] = []
    name_to_printings[name].append({
        'printing_id': card['printing_unique_id'],
        'set_id': card.get('set_id', ''),
        'foiling': card.get('foiling', ''),
        'phash': card.get('image_phash', '')[:64]
    })

print(f"✓ Loaded {len(all_cards):,} cards, {len(name_to_printings):,} unique names")

In [ ]:
class TwoStageIdentifier:
    def __init__(self, model, ref_embeddings, ref_names, card_lookup, name_to_printings, device):
        self.model = model
        self.ref_embeddings = ref_embeddings
        self.ref_names = ref_names
        self.card_lookup = card_lookup
        self.name_to_printings = name_to_printings
        self.device = device
        self.transform = get_val_transforms()
    
    def identify(self, image_input):
        if isinstance(image_input, str):
            pil_img = Image.open(image_input).convert('RGB')
        else:
            pil_img = Image.fromarray(image_input) if isinstance(image_input, np.ndarray) else image_input
        
        pil_img = ImageOps.autocontrast(pil_img, cutoff=1)
        img_array = np.array(pil_img)
        
        # Stage 1: CNN
        with torch.no_grad():
            tensor = self.transform(image=img_array)['image'].unsqueeze(0).to(self.device)
            query_emb = self.model(tensor).cpu()
        
        sims = F.cosine_similarity(query_emb, self.ref_embeddings)
        top_idx = sims.argsort(descending=True)[:5]
        
        top_id = self.ref_names[top_idx[0]]
        card_info = self.card_lookup.get(top_id, {})
        card_name = card_info.get('name', 'Unknown')
        
        result = {'name': card_name, 'cnn_confidence': sims[top_idx[0]].item()}
        
        # Stage 2: pHash
        query_phash = imagehash.phash(pil_img, hash_size=16)
        printings = self.name_to_printings.get(card_name, [])
        
        if printings:
            best = min(printings, key=lambda p: 
                       (imagehash.hex_to_hash(p['phash']) - query_phash) if p['phash'] else 999)
            result.update({'printing_id': best['printing_id'], 'set_id': best['set_id'], 
                          'foiling': best['foiling']})
        
        return result

identifier = TwoStageIdentifier(model, reference_embeddings, reference_names, 
                                 card_lookup, name_to_printings, device)
print('✓ TwoStageIdentifier ready!')

In [ ]:
from google.colab import files

print('Upload card images to test...')
uploaded = files.upload()

for filename in uploaded.keys():
    result = identifier.identify(filename)
    print(f'\n🎴 {result["name"]}')
    print(f'   CNN: {result["cnn_confidence"]*100:.1f}%')
    print(f'   Printing: {result.get("set_id", "?")} ({result.get("foiling", "?")})')
    
    plt.figure(figsize=(5,7))
    plt.imshow(Image.open(filename))
    plt.title(f'{result["name"]} ({result["cnn_confidence"]*100:.0f}%)')
    plt.axis('off')
    plt.show()

## ✅ Done!

**Curriculum Learning Phases:**
- Phase 1 (1-10): Light augmentation
- Phase 2 (11-20): Medium augmentation
- Phase 3 (21+): Heavy + backgrounds

**Live Monitoring:** Top-1/5/10 accuracy every 5 epochs